# Majority Filter
Cascade Tuholske, Dec 2020<br>

Notebook is to pass a majority filter on MGUP modis data to resample it at 1-km instead of 250m.<br>
https://doi.org/10.1016/j.jag.2020.102255

In [15]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio 
import rioxarray as rio
from glob import glob
import skimage
from skimage.filters.rank import majority

In [16]:
# file paths and FN
data_in = '/Users/cascade/Github/PopGridCompare/data/raw/'
data_out = '/Users/cascade/Github/PopGridCompare/data/interim/'

mgup_in_fn = data_in+'MGUP_annual_2001_2018/MGUP_2015.tif'
mgup_out_fn = data_out+'MGUP_2015-1km.tif'

In [17]:
mgup_in = rasterio.open(mgup_in_fn)

In [18]:
# 'width': 160296, 'height': 63965, epsg:4326
mgup_in.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 160296,
 'height': 63965,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.002245788210298804, 0.0, -179.9909419026068,
        0.0, -0.002245788210298804, 84.1294721460035)}

In [21]:
# Re sample with rasterio 
import rasterio
from rasterio.enums import Resampling

scale_factor = 0.5 # 500m to 1-km

with rasterio.open(mgup_in_fn) as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * scale_factor),
            int(dataset.width * scale_factor)
        ),
        resampling=Resampling.mode
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [23]:
mgup_in_fn

'/Users/cascade/Github/PopGridCompare/data/raw/MGUP_annual_2001_2018/MGUP_2015.tif'

In [20]:
# Save it out
out_meta = mgup_in.meta

# Update meta data
out_meta.update({"driver": "GTiff",
             "height": data.shape[1],
             "width": data.shape[2],
             "transform": transform})

# write image 
with rasterio.open(mgup_out_fn, "w", **out_meta) as dest:
    dest.write(data)